# Femnist Table 1

TJ Kim

1.17.22

#### Summary:
- Make a table for Benign transferability and inter-boundary distance for following models
- Local benign, fedavg benign, fedEM benign

In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from torch.utils.tensorboard import SummaryWriter
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *

from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

#### Local Benign

In [3]:
setting = 'local'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "femnist"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=300)

==> Clients initialization..
===> Building data iterators..


100%|██████████| 719/719 [00:01<00:00, 462.58it/s]


===> Initializing clients..


100%|██████████| 719/719 [00:08<00:00, 80.11it/s] 


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.147 | Train Acc: 1.769% |Test Loss: 4.146 | Test Acc: 1.699% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


Generate Dataset to be used throughout all analysis.

In [4]:
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].val_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 

Load local model.

In [28]:
num_models = 8

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

    args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='weights/final/cifar/fig3/local_defend'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
    weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
#     weights = np.load('weights/final/cifar/fig3/local_defend/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
    args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
#     args_.save_path = 'weights/final/cifar/fig3/fed_avg_defend'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
#     weights = np.load('weights/final/cifar/fig3/fed_avg_defend/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
    args_.save_path = 'weights/final/femnist/fig1_take3/fedem_benign/'
#     args_.save_path = 'weights/final/cifar/fig3/fedem_defend/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

    weights = np.load("weights/final/femnist/fig1_take3/fedem_benign/train_client_weights.npy")
#     weights = np.load("weights/final/cifar/fig3/fedem_defend/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0,w1,w2) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0*weights_h[0][key] + w1*weights_h[1][key] + w2*weights_h[2][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

In [29]:
weights

array([[1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.00],
       [1.

Set Up Transfer Attack Scenario

In [30]:
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]


In [32]:
# Run Measurements for both targetted and untargeted analysis
new_num_models = 8
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 6.5


for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    dataloader = load_client_data(clients = clients, c_id = adv_idx, mode = 'test') # or test/train
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
    
    t1 = Transferer(models_list=models_test, dataloader=dataloader)
    t1.generate_victims(victim_idxs)
    
    # Perform Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 50,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    
    
    
    t1.generate_advNN(adv_idx)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack
    t1.atk_params.set_params(batch_size=batch_size, iteration = 50,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)

	 Adv idx: 0
	 Adv idx: 1
	 Adv idx: 2
	 Adv idx: 3
	 Adv idx: 4
	 Adv idx: 5
	 Adv idx: 6
	 Adv idx: 7


Print Relevant Information

In [33]:
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)])
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) 

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [34]:
print('adv_target:', avg_nondiag(adv_target))
print('adv_miss:', avg_nondiag(adv_miss))
print('orig_acc:', np.mean(np.diagonal(orig_acc)))

adv_target: 0.043812112450333576
adv_miss: 0.17166360254798615
orig_acc: 0.6152382865548134


In [36]:
orig_acc

array([[0.68, 0.29, 0.56, 0.29, 0.29, 0.56, 0.35, 0.50],
       [0.51, 0.62, 0.59, 0.27, 0.68, 0.16, 0.30, 0.27],
       [0.50, 0.35, 0.59, 0.09, 0.47, 0.26, 0.41, 0.38],
       [0.13, 0.17, 0.13, 0.66, 0.08, 0.28, 0.20, 0.23],
       [0.28, 0.53, 0.34, 0.06, 0.56, 0.06, 0.12, 0.03],
       [0.30, 0.11, 0.23, 0.37, 0.11, 0.63, 0.28, 0.32],
       [0.20, 0.02, 0.18, 0.33, 0.10, 0.37, 0.63, 0.45],
       [0.09, 0.08, 0.09, 0.25, 0.08, 0.30, 0.23, 0.55]])

Set Up Interboundary Measure Scenario

In [37]:
dataloader.y_data.shape

torch.Size([64])

In [41]:
num_trials = 50
batch_size = 5000
adv_idx = [0]

dataloader = Custom_Dataloader(data_x, data_y)

# t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
# t1.base_nn_idx = 0
# t1.victim_idx = [5]
vidx = [1,2,3,4,5,6,7]

dists_measure_legit = np.zeros([num_trials, len(vidx)])
dists_measure_adv = np.zeros([num_trials, len(vidx)])
dists_measure_adv_ensemble = np.zeros([num_trials, len(vidx)])

for i in range(num_trials):
    print("num_trial:", i)
    t1 = Boundary_Transferer(models_list=models_test, dataloader=dataloader)
    t1.base_nn_idx = 0
    t1.victim_idx = vidx

    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=500, iteration = 30,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.05, step_norm = "inf", eps = 3, eps_norm = 2)
    t1.set_adv_NN(t1.base_nn_idx)

    base_ep_legit, victim_eps_legit = t1.legitimate_direction(batch_size=batch_size, ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = True,print_res = False)
    
    base_ep_adv, victim_eps_adv = t1.adversarial_direction(ep_granularity = 0.5, 
                                                              rep_padding = 1000, new_point = False,print_res = False)
    
    idx = 0
    for key, value in victim_eps_legit.items():
        dists_measure_legit[i,idx] = np.abs(base_ep_legit-value)
        idx+=1
        
    idx = 0
    for key, value in victim_eps_adv.items():
        dists_measure_adv[i,idx] = np.abs(base_ep_adv - value)
        idx+=1
    

num_trial: 0
num_trial: 1
num_trial: 2
num_trial: 3
num_trial: 4
num_trial: 5
num_trial: 6
num_trial: 7
num_trial: 8
num_trial: 9
num_trial: 10
num_trial: 11
num_trial: 12
num_trial: 13
num_trial: 14
num_trial: 15
num_trial: 16
num_trial: 17
num_trial: 18
num_trial: 19
num_trial: 20
num_trial: 21
num_trial: 22
num_trial: 23
num_trial: 24
num_trial: 25
num_trial: 26
num_trial: 27
num_trial: 28
num_trial: 29
num_trial: 30
num_trial: 31
num_trial: 32
num_trial: 33
num_trial: 34
num_trial: 35
num_trial: 36
num_trial: 37
num_trial: 38
num_trial: 39
num_trial: 40
num_trial: 41
num_trial: 42
num_trial: 43
num_trial: 44
num_trial: 45
num_trial: 46
num_trial: 47
num_trial: 48
num_trial: 49


In [42]:
np.mean(np.average(dists_measure_legit,axis=0))

10.341428571428569

In [43]:
np.mean(np.average(dists_measure_adv,axis=0))

45.61142857142857

#### Logs

In [ ]:
# Global Validation Dataset

# Local - Benign
local_adv_miss = 0.21# temp
local_adv_target = 0.03#temp
local_orig_acc = 0.61
local_ibdist_legit = 7.615
local_ibdist_adv = 41.505625

# Fedavg - Benign
fedavg_adv_miss = 0.34 # temp
fedavg_adv_target = 0.10 # temp
fedavg_orig_acc = 0.72
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0 # 15.55

# FedEM - Benign
fedem_adv_miss = 0.33 # temp
fedem_adv_target = 0.10 # temp
fedem_orig_acc = 0.67
fedem_ibdist_legit = 0.01125
fedem_ibdist_adv = 13.613125

# Local - adv
local_adv_miss = 
local_adv_target = 
local_orig_acc = 
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 
fedavg_adv_target =  
fedavg_orig_acc = 
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 
fedem_adv_target = 
fedem_orig_acc = 
fedem_ibdist_legit = 
fedem_ibdist_adv = 

In [ ]:
# Local Validation Dataset

# Local
local_adv_miss = 0.37
local_adv_target = 0.06
local_orig_acc = 0.99
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.94
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.44
fedem_orig_acc = 0.94
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.29
local_adv_target = 0.05
local_orig_acc = 0.77
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.33
fedavg_adv_target =  0.24
fedavg_orig_acc = 0.87
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.49
fedem_adv_target = 0.07
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

In [ ]:
# Local Test (Train) Dataset

# Local
local_adv_miss = 0.38
local_adv_target = 0.06
local_orig_acc = 0.52
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - Benign
fedavg_adv_miss = 0.00
fedavg_adv_target = 0.85
fedavg_orig_acc = 0.81
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - Benign
fedem_adv_miss = 0.10
fedem_adv_target = 0.46
fedem_orig_acc = 0.84
fedem_ibdist_legit = 
fedem_ibdist_adv = 

# Local - adv
local_adv_miss = 0.30
local_adv_target = 0.05
local_orig_acc = 0.46
local_ibdist_legit = 
local_ibdist_adv = 

# Fedavg - adv
fedavg_adv_miss = 0.26
fedavg_adv_target = 0.30
fedavg_orig_acc = 0.74
fedavg_ibdist_legit = 0
fedavg_ibdist_adv = 0

# FedEM - adv
fedem_adv_miss = 0.42
fedem_adv_target = 0.09
fedem_orig_acc = 0.74
fedem_ibdist_legit = 
fedem_ibdist_adv = 